In [43]:
## Import libraries
import requests
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [44]:
# ## Connect to MySQL DB - LOCAL   
# driver   = 'mysql+pymysql:'
# user     = 'root' ## the user you use to access the DB
# password = 'ML_LI_04' ## the password you use to access the DB
# ip       = '127.0.0.1' ## the DB host IP
# database = 'data_thieves_dummy' ## name_of_DB_you_want_to_connect

## Connect to MySQL DB - CLOUD   
driver   = 'mysql+pymysql:'
user     = 'root' ## the user you use to access the DB
password = 'ML_LI_04' ## the password you use to access the DB
ip       = '35.195.172.42' ## the DB host IP
database = 'data_thieves' ## name_of_DB_you_want_to_connect

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

### Import and normalize artist table

In [45]:
##Read table
artists_table = pd.read_sql('artist', engine)
artists_table.head(3)

,ID,artist_name
0,1,Henri Salvador
1,2,Martin & les fées
2,3,Joseph Williams


In [46]:
##Rename columns
artists_table = artists_table.rename(columns={'artist_name':'artist'})
artists_table = artists_table.rename(columns={'ID':'artist_id'})
artists_table.head(3)

,artist_id,artist
0,1,Henri Salvador
1,2,Martin & les fées
2,3,Joseph Williams


### Import and normalize country table

In [47]:
##Read table
country_table = pd.read_sql('country', engine)
country_table.head(3)

,ID,country_name,alpha_2_code
0,1,Afghanistan,AF
1,2,Åland Islands,AX
2,3,Albania,AL


In [48]:
country_table = country_table.rename(columns={'ID':'country_id'})
country_table.head(3)

,country_id,country_name,alpha_2_code
0,1,Afghanistan,AF
1,2,Åland Islands,AX
2,3,Albania,AL


### Wrangling Jordan's top artist list

In [49]:
top_artist_jordan = pd.read_csv('top_artist_jordan.csv')
top_artist_jordan.head(3)

,Unnamed: 0,rank,artist,country
0,0,1,Lana Del Rey,Jordan
1,1,2,Pink Floyd,Jordan
2,2,3,Queen,Jordan


In [11]:
## Check country name in country table
country_table[country_table['country_name'] == 'Jordan']

,country_id,country_name,alpha_2_code
111,112,Jordan,JO


In [12]:
##Find missing artists
artist_jordan_list = list(top_artist_jordan.artist)
artists_table_list = list(artists_table.artist)
for i in artist_jordan_list:
    if i not in artists_table_list:
        print(i)

Florence + the Machine
System of a Down
Fairuz
Frank Sinatra


### Wrangling Switzerland's top artitst

In [50]:
top_artist_switzerland = pd.read_csv('top_artist_switzerland.csv')
top_artist_switzerland.head(3)

,Unnamed: 0,rank,artist,country
0,0,1,Coldplay,Switzerland
1,1,2,Queen,Switzerland
2,2,3,David Bowie,Switzerland


In [14]:
## Check country name in country table
country_table[country_table['country_name'] == 'Switzerland']

,country_id,country_name,alpha_2_code
212,213,Switzerland,CH


In [15]:
##Find missing
artist_switzerland_list = list(top_artist_switzerland.artist)
artists_table_list = list(artists_table.artist)
for i in artist_switzerland_list:
    if i not in artists_table_list:
        print(i)

Florence + the Machine
Mumford & Sons


### Wrangling Spain's top artitst

In [51]:
top_artist_spain = pd.read_csv('top_artist_spain.csv')
top_artist_spain.head(3)

,Unnamed: 0,rank,artist,country
0,0,1,Radiohead,Spain
1,1,2,David Bowie,Spain
2,2,3,Kanye West,Spain


In [17]:
## Check country name in country table
country_table[country_table['country_name'] == 'Spain']

,country_id,country_name,alpha_2_code
205,206,Spain,ES


In [18]:
##Find missing
artist_spain_list = list(top_artist_spain.artist)
artists_table_list = list(artists_table.artist)
for i in artist_spain_list:
    if i not in artists_table_list:
        print(i)

Florence + the Machine


### Find parcial coincidences in artist table for missing artists:

* Florence + the Machine
* System of a Down
* Fairuz
* Frank Sinatra
* Mumford & Sons

In [19]:
## Florence + the Machine
artists_table.loc[artists_table.artist.str.contains("Florence")]

,artist_id,artist
457,458,Florence + The Machine
9168,9169,Florence Beatrice Price


In [20]:
## System of a Down
artists_table.loc[artists_table.artist.str.contains("ystem")]

,artist_id,artist
409,410,System Of A Down
657,658,LCD Soundsystem
9766,9767,Sabotage Soundsystem
9917,9918,PapaShanty SaundSystem
13959,13960,Afro Celt Sound System
13977,13978,EarthRise SoundSystem


In [21]:
## Fairuz
artists_table.loc[artists_table.artist.str.contains("iru")]

,artist_id,artist
3795,3796,Michiru Oshima
4221,4222,Beirut
6101,6102,Pirulo y la Tribu
9531,9532,Natiruts


In [22]:
## Frank Sinatra
artists_table.loc[artists_table.artist.str.contains("inatra")]

,artist_id,artist
4310,4311,Nancy Sinatra


In [23]:
## Mumford & Sons
artists_table.loc[artists_table.artist.str.contains("Sons")]

,artist_id,artist
1229,1230,Sons Of Bill
1272,1273,Rival Sons
4118,4119,Sons Of The East
6952,6953,The Glorious Sons
7954,7955,Young Rising Sons
8859,8860,Sons Of Zion
9239,9240,Sons Of Apollo
10851,10852,Summers Sons
11176,11177,Sons Of Kemet
11702,11703,The Arrogant Sons Of Bitches


Two actions to be done:

* Add the missing artists to the artist table in our DB
* Rename the partially matching artists on the country top artists DataFrames

In [52]:
##Edit the Top Artist CSVs to match the artist name 
top_artist_jordan.loc[top_artist_jordan['artist'] == 'Florence + the Machine', 'artist'] = 'Florence + The Machine'
top_artist_switzerland.loc[top_artist_switzerland['artist'] == 'Florence + the Machine', 'artist'] = 'Florence + The Machine'
top_artist_spain.loc[top_artist_spain['artist'] == 'Florence + the Machine', 'artist'] = 'Florence + The Machine'
top_artist_jordan.loc[top_artist_jordan['artist'] == 'System of a Down', 'artist'] = 'System Of A Down'

In [25]:
##Create DataFrame with the missing artists
new_artists = ['Mumford & Sons', 'Fairuz', 'Frank Sinatra']
df_new_artists = pd.DataFrame(new_artists)
df_new_artists.columns = ['artist_name']

In [26]:
##Add the missing artists to the DB table
df_new_artists.to_sql('artist', con=engine, if_exists = 'append', index = False) 

### Create new top charts table

In [53]:
top_artists = pd.concat([top_artist_jordan, top_artist_switzerland, top_artist_spain], axis=0)
top_artists.drop('Unnamed: 0', axis=1, inplace = True)

In [54]:
##Add the country_id to the top_artists table
top_artists = top_artists.rename(columns={'country':'country_name'})
top_artists = pd.merge(top_artists, country_table, on='country_name')
top_artists.drop('alpha_2_code', axis=1, inplace = True)

In [55]:
##Add the artist_id to the top_artists table
##Import table with new added artists
new_artists_table = pd.read_sql('artist', engine)

In [56]:
new_artists_table.head(3)

,ID,artist_name
0,1,Henri Salvador
1,2,Martin & les fées
2,3,Joseph Williams


In [57]:
top_artists = top_artists.rename(columns={'artist':'artist_name'})

In [58]:
new_artists_table = new_artists_table.rename(columns={'ID':'artist_id'})

In [59]:
top_artists = pd.merge(top_artists, new_artists_table, on='artist_name')
top_artists.head(3)

,rank,artist_name,country_name,country_id,artist_id
0,1,Lana Del Rey,Jordan,112,2358
1,20,Lana Del Rey,Switzerland,213,2358
2,18,Lana Del Rey,Spain,206,2358


In [60]:
top_artists.columns = ['ranking', 'artist_name', 'country_name', 'country_id', 'artist_id']

In [61]:
## Export it to MySQL
top_artists.to_sql('top_artists_country', con=engine, if_exists = 'append', index = False) 

In [62]:
##Import it back with the table PK
top_artists_with_id = pd.read_sql('top_artists_country', engine)
print('Len top artists table after DB: ', len(top_artists_with_id))

Len top artists table after DB:  150
